In [62]:
import datetime
import pandas as pd

import pickle

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing

In [63]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [64]:
MODEL_FOLDER = "model"

PROCESSED_DATA_FOLDER = "data/4_all_data_preprocessed"
ISW_DATA_FILE = "all_isw.csv"
WEATHER_EVENTS_DATA_FILE = "features_generated_v1.csv"

OUTPUT_DATA_FILE = "all_features"

tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v4"
count_vectorizer_version = "v4"

In [65]:
def isNaN(num):
    return num != num

### Load preprocessed data

In [66]:
# Load CSVs
df_isw = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{ISW_DATA_FILE}", sep=";")
df_weather_events = pd.read_csv(f"{PROCESSED_DATA_FOLDER}/{WEATHER_EVENTS_DATA_FILE}", sep=";")

df_weather_events = df_weather_events.loc[:, ~df_weather_events.columns.str.contains('^Unnamed')]

print(df_isw.shape)

# Load TF-IDF
tfidf = pickle.load(open(f"{MODEL_FOLDER}/{tfidf_transformer_model}_{tfidf_transformer_version}.pkl", "rb"))
cv = pickle.load(open(f"{MODEL_FOLDER}/{count_vectorizer_model}_{count_vectorizer_version}.pkl", "rb"))

(402, 7)


### Prepare weather & events

In [67]:
# Technicaly it's feature engineering

# Replace timestamps with numerical values
df_weather_events.day_sunset = df_weather_events.day_sunset.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.day_sunrise = df_weather_events.day_sunrise.apply(lambda x: 
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()
)
df_weather_events.hour_datetime = df_weather_events.hour_datetime.apply(lambda x:
    (datetime.datetime.strptime(x, "%H:%M:%S") - datetime.datetime.strptime("00:00:00", "%H:%M:%S")).total_seconds()//3600
)

# Encode categories
le = preprocessing.LabelEncoder()
df_weather_events.hour_conditions = le.fit_transform(df_weather_events.hour_conditions)
df_weather_events.hour_preciptype = le.fit_transform(df_weather_events.hour_preciptype)

# Create new column with boolean event indicator
df_weather_events["event_indicator"] = df_weather_events.event_hour_level_event_datetimeEpoch.apply(lambda x: not isNaN(x))

# Drop now irrelevant features
features_to_drop = [
    "day_datetimeEpoch",
    "hour_datetimeEpoch",
    "event_all_region",
    "event_clean_end",
    "event_intersection_alarm_id",
    "event_start_time",
    "event_end_time",
    "event_start_hour",
    "event_end_hour",
    "event_day_date",
    "event_start_hour_datetimeEpoch",
    "event_end_hour_datetimeEpoch",	
    "event_hour_level_event_time",
    "event_hour_level_event_datetimeEpoch",
    "event_region_title",
    "event_region_city",
    "event_start",
    "event_end",
    "city",
    "region",
    "center_city_ua",
    "center_city_en",
    "region_alt"
]
df_weather_events = df_weather_events.drop(features_to_drop, axis=1)

# Convert everything to float
df_weather_events.hour_preciptype = df_weather_events.hour_preciptype.astype(float)
df_weather_events.hour_conditions = df_weather_events.hour_conditions.astype(float)
df_weather_events.region_id = df_weather_events.region_id.astype(float)
df_weather_events.event_alarms_past_24 = df_weather_events.event_alarms_past_24.astype(float)
df_weather_events.event_simultaneous_alarms = df_weather_events.event_simultaneous_alarms.astype(float)

df_weather_events.event_holiday_is_near = df_weather_events.event_holiday_is_near.apply(lambda x: 1.0 if x and not isNaN(x) else 0.0)

# Fill NAN
df_weather_events.fillna(0, inplace=True)

print(df_weather_events.dtypes)
df_weather_events.sample(10)

city_resolvedAddress            object
day_datetime                    object
day_tempmax                    float64
day_tempmin                    float64
day_temp                       float64
day_dew                        float64
day_humidity                   float64
day_precip                     float64
day_precipcover                float64
day_solarradiation             float64
day_solarenergy                float64
day_uvindex                    float64
day_sunrise                    float64
day_sunset                     float64
day_moonphase                  float64
hour_datetime                  float64
hour_temp                      float64
hour_humidity                  float64
hour_dew                       float64
hour_precip                    float64
hour_precipprob                float64
hour_snow                      float64
hour_snowdepth                 float64
hour_preciptype                float64
hour_windgust                  float64
hour_windspeed           

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,event_indicator
148184,"Полтава, Україна",2022-05-17,18.1,7.3,13.3,0.9,44.7,0.000,0.00,285.3,24.7,9.0,17706.0,73332.0,0.54,9.0,14.2,37.51,-0.1,0.0,0.0,0.0,0.0,5.0,25.2,14.4,290.0,1013.9,10.0,30.0,517.0,1.9,5.0,10.0,4.0,16.0,2.0,0.0,3.074444,0.0,True
143177,"Тернопіль, Україна",2022-10-01,15.7,11.1,13.8,12.5,91.8,6.000,8.33,27.9,2.4,2.0,26226.0,68206.0,0.17,0.0,14.2,96.18,13.6,0.0,0.0,0.0,0.0,5.0,15.8,3.6,230.0,1014.8,10.0,80.0,0.0,0.0,0.0,10.0,4.0,19.0,2.0,1.0,2.404444,0.0,True
14036,"Кропивницький, Україна",2022-10-11,15.4,3.7,8.6,2.2,67.1,0.000,0.00,160.3,13.8,6.0,25471.0,65408.0,0.51,9.0,4.7,85.04,2.4,0.0,0.0,0.0,0.0,5.0,27.0,14.4,320.0,1023.4,10.0,0.0,177.0,0.6,2.0,10.0,0.0,11.0,6.0,24.0,6.950000,1.0,False
123734,"Рівне, Україна",2022-06-08,28.0,13.8,21.8,10.4,51.5,0.000,0.00,335.5,28.9,9.0,18238.0,77077.0,0.28,8.0,19.6,62.31,12.2,0.0,0.0,0.0,0.0,5.0,15.1,8.3,136.1,1012.0,24.1,38.5,271.0,1.0,3.0,10.0,4.0,17.0,1.0,9.0,11.145833,0.0,False
122412,"Рівне, Україна",2022-04-14,16.2,-1.1,8.1,-1.7,54.1,0.000,0.00,252.1,21.9,8.0,23008.0,72494.0,0.43,14.0,14.0,38.00,-0.1,0.0,0.0,0.0,0.0,5.0,9.7,3.2,8.1,1022.0,24.1,4.6,767.0,2.8,8.0,10.0,0.0,17.0,0.0,8.0,14.811389,1.0,False
29169,"Київ, Україна",2022-04-27,17.2,7.2,11.8,4.1,62.5,0.400,4.17,189.8,16.5,7.0,20444.0,72690.0,0.87,19.0,14.0,42.04,1.3,0.0,0.0,0.0,0.0,5.0,23.0,6.8,319.0,1020.0,24.1,71.5,113.0,0.4,1.0,10.0,4.0,10.0,4.0,8.0,0.290833,1.0,False
1333,"Луцьк, Луцький район, Україна",2022-04-17,8.3,1.4,4.3,-2.9,61.7,0.300,12.50,217.2,18.8,6.0,22839.0,73021.0,0.53,9.0,2.9,76.08,-0.9,0.0,0.0,0.0,0.0,5.0,33.1,20.1,16.0,1025.4,14.1,65.5,242.0,0.9,2.0,10.0,4.0,3.0,3.0,0.0,2.157222,0.0,True
105627,"Ужгород, Ужгородський район, Україна",2022-04-05,8.8,1.1,4.2,-0.4,72.5,0.685,8.33,35.8,3.3,1.0,25251.0,72437.0,0.12,10.0,3.2,71.51,-1.5,0.0,0.0,0.0,0.0,5.0,31.7,18.0,196.0,1008.1,17.0,93.1,68.0,0.2,1.0,10.0,3.0,7.0,3.0,2.0,3.739722,0.0,True
113730,"Запоріжжя, Україна",2022-04-08,15.0,5.7,10.5,5.3,71.8,0.800,12.50,112.5,9.9,6.0,21857.0,69549.0,0.22,10.0,12.4,61.47,5.2,0.0,0.0,0.0,0.0,5.0,22.7,11.2,123.8,999.0,24.1,17.7,521.0,1.9,5.0,10.0,0.0,8.0,5.0,1.0,1.259722,1.0,True
6132,"Луцьк, Луцький район, Україна",2022-11-02,12.0,9.4,10.8,9.6,92.3,0.200,8.33,57.6,4.9,3.0,25898.0,60730.0,0.25,4.0,10.3,93.52,9.3,0.0,0.0,0.0,0.0,5.0,23.4,13.0,293.9,1017.0,24.1,100.0,0.0,0.0,0.0,10.0,3.0,3.0,1.0,0.0,17.281944,0.0,False


### Calculate TF-IDF matrix

In [68]:
# Calculate word count vector for each report (save as matrix)
word_count_vector = cv.transform(df_isw["lemm"].values.astype("U"))
# print("Word count vector:\n", word_count_vector)

# Calculate global tfidf vectors matrix
tfidf_vector = tfidf.transform(word_count_vector)

# transform tfidf matrix to sparse dataframe
df_tfidf_vector = pd.DataFrame.sparse.from_spmatrix(tfidf_vector, columns=cv.get_feature_names_out())
df_tfidf_vector = pd.concat([df_isw.date_tomorrow_datetime, df_tfidf_vector], axis=1)

print(df_tfidf_vector.shape)

(402, 719)


### Merge final dataset

In [69]:
# Merge weather events dataset with yesterday report tfidf matrix (takes 2m to execute)
df_final = df_weather_events.merge(df_tfidf_vector.add_prefix("isw_"),
                                    how="inner",
                                    left_on="day_datetime",
                                    right_on="isw_date_tomorrow_datetime")

# Reorder columns
df_final.insert(2, 'isw_date_tomorrow_datetime', df_final.pop('isw_date_tomorrow_datetime'))
df_final.insert(3, 'event_indicator', df_final.pop('event_indicator'))

print(df_final.shape)
df_final.sample(5)

(192867, 760)


,city_resolvedAddress,day_datetime,isw_date_tomorrow_datetime,event_indicator,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,day_sunrise,day_sunset,day_moonphase,hour_datetime,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_preciptype,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_solarenergy,hour_uvindex,hour_severerisk,hour_conditions,region_id,event_alarms_past_24,event_simultaneous_alarms,event_hours_from_last_alarm,event_holiday_is_near,isw_10km,isw_12km,isw_15km,isw_1st,isw_20km,isw_30km,isw_5km,isw_6km,isw_abandon,isw_abil,isw_abl,isw_access,isw_accus,isw_achiev,isw_across,isw_action,isw_activ,isw_ad,isw_addit,isw_address,isw_administr,isw_advanc,isw_advisor,isw_affect,isw_affili,isw_aim,isw_air,isw_airborn,isw_aircraft,isw_airstrik,isw_alexand,isw_allow,isw_almost,isw_along,isw_alongsid,isw_alreadi,isw_also,isw_although,isw_ammunit,isw_among,isw_amplifi,isw_annex,isw_announc,isw_anoth,isw_anti,isw_appar,isw_appeal,isw_appear,isw_appoint,isw_approxim,isw_april,isw_area,isw_arm,isw_armi,isw_armor,isw_around,isw_arrang,isw_arriv,isw_artilleri,isw_assault,isw_asset,isw_attempt,isw_author,isw_avail,isw_avdiivka,isw_avoid,isw_away,isw_axi,isw_back,isw_bakhmut,isw_bank,isw_base,isw_basi,isw_battalion,isw_battl,isw_becom,isw_began,isw_begin,isw_belaru,isw_belarusian,isw_belgorod,isw_bilohorivka,isw_blame,isw_border,isw_break,isw_bridg,isw_brigad,isw_build,isw_call,isw_campaign,isw_capabl,isw_captur,isw_carri,isw_case,isw_casualti,isw_cauldron,isw_caus,isw_center,isw_central,isw_challeng,isw_chang,isw_channel,isw_chechen,isw_chief,isw_child,isw_choos,isw_citizen,isw_civilian,isw_classifi,isw_clear,isw_click,isw_close,isw_colonel,isw_combat,isw_combin,isw_come,isw_command,isw_comment,isw_commerci,isw_commit,isw_commun,isw_compani,isw_complet,isw_compris,isw_concentr,isw_concern,isw_condemn,isw_condit,isw_confirm,isw_conflict,isw_connect,isw_conscript,isw_consid,isw_consist,isw_consolid,isw_construct,isw_contact,isw_contract,isw_control,isw_convent,isw_corp,isw_could,isw_council,isw_counter,isw_counterattack,isw_counteroffens,isw_countri,isw_cours,isw_cover,isw_creat,isw_crime,isw_crimea,isw_crimin,isw_critic,isw_cross,isw_current,isw_cut,isw_daili,isw_damag,isw_data,isw_date,isw_day,isw_decemb,isw_decis,isw_defend,isw_defens,isw_degrad,isw_demand,isw_demonstr,isw_deni,isw_deploy,isw_depot,isw_deputi,isw_describ,isw_despit,isw_destroy,isw_detail,isw_develop,isw_differ,isw_direct,isw_directli,isw_director,isw_discus,isw_disrupt,isw_district,isw_divis,isw_dmitri,isw_dnipro,isw_dnipropetrovsk,isw_dnr,isw_domest,isw_donba,isw_draw,isw_drive,isw_drone,isw_due,isw_duma,isw_earli,isw_east,isw_eastern,isw_econom,isw_effect,isw_eight,isw_eighteen,isw_eighti,isw_either,isw_element,isw_eleven,isw_emphas,isw_encircl,isw_end,isw_endnot,isw_energi,isw_enerhodar,isw_enforc,isw_engag,isw_enter,isw_entir,isw_entireti,isw_equip,isw_establish,isw_evacu,isw_evalu,isw_even,isw_evid,isw_exist,isw_expand,isw_expect,isw_explos,isw_express,isw_extens,isw_face,isw_facil,isw_fail,isw_failur,isw_fall,isw_fals,isw_famili,isw_far,isw_februari,isw_feder,isw_fifteen,isw_fifti,isw_fight,isw_fighter,isw_financi,isw_fire,isw_first,isw_five,isw_focu,isw_focus,isw_follow,isw_footag,isw_forcibl,isw_forecast,isw_foreign,isw_form,isw_format,isw_former,isw_forti,isw_fortif,isw_four,isw_fourteen,isw_front,isw_frontlin,isw_full,isw_fulli,isw_futur,isw_gain,isw_geneva,isw_geoloc,isw_geospati,isw_given,isw_gloc,isw_govern,isw_governor,isw_ground,isw_group,isw_grow,isw_guard,isw_gur,isw_haidai,isw_head,isw_heavi,isw_heavili,isw_held,isw_high,isw_highli,isw_highway,isw_himar,isw_hold,isw_hope,isw_hour,isw_howev,isw_hulyaipol,isw_human,isw_hundr,isw_hundred,isw_identifi,isw_imageri,isw_immedi,isw_immin,isw_import,isw_improv,isw_includ,isw_increas,isw_increasingli,isw_independ,isw_indic,isw_

In [70]:
# Save final dataset
with open(f"{PROCESSED_DATA_FOLDER}/{OUTPUT_DATA_FILE}.pkl", 'wb') as handle:
    pickle.dump(df_final, handle)

In [73]:
"""y = df_final.event_indicator.astype(float)
df_final = df_final.drop(["isw_date_tomorrow_datetime", "day_datetime", "city_resolvedAddress", "event_indicator"], axis=1)

y.head(5)"""

count    192867.000000
mean          0.254548
std           0.435608
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: event_indicator, dtype: float64